# LangGraph Chatbot with Message History

This notebook demonstrates how to build a conversational chatbot using LangGraph that maintains conversation history and context across multiple interactions.

## Step 1: Load Environment Variables

Load environment variables from `.env` file to access API keys (like Google Gemini API key) securely.

In [4]:
from dotenv import load_dotenv

load_dotenv()

True

## Step 2: Import Required Libraries

Import necessary modules:
- `init_chat_model`: For initializing the LLM
- `TypedDict`, `Annotated`: For type definitions
- `StateGraph`, `START`, `END`: For building the graph
- `add_messages`: A reducer function that appends messages to the conversation history

In [17]:
from langchain.chat_models import init_chat_model
from typing import TypedDict, Literal, Annotated
from langgraph.graph import StateGraph, START, END
from IPython.display import display, Image
from langgraph.graph.message import add_messages

## Step 3: Initialize the Language Model

Create an instance of Google's Gemini Flash Lite model and test it with a simple query to verify it's working correctly.

In [7]:
llm = init_chat_model("google_genai:gemini-2.5-flash-lite")
llm.invoke("Who won the world series in 2023?")

AIMessage(content='The **Texas Rangers** won the World Series in 2023.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019af760-273d-7762-a4eb-564633009b82-0', usage_metadata={'input_tokens': 13, 'output_tokens': 16, 'total_tokens': 29, 'input_token_details': {'cache_read': 0}})

## Step 4: Define State and Chatbot Function

Define the conversation state structure:
- `State`: Contains a `messages` list with the `add_messages` reducer to accumulate conversation history
- `chatbot()`: A function that takes the current state, invokes the LLM with the message history, and returns the response

In [18]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

## Step 5: Build and Compile the Graph

Create a simple linear graph:
- Single node: `chatbot` (processes messages and generates responses)
- Flow: START → chatbot → END
- The graph maintains conversation state across invocations

In [19]:
builder = StateGraph(State)

builder.add_node("chatbot", chatbot)

builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

graph = builder.compile()

## Step 6: Test with a Single Message

Test the chatbot with a single question to verify it's working. The response will include both the user message and the AI's response in the messages list.

In [20]:
message= {"role": "user", "content": "Who is the first person to walk on the moon?"}
response = graph.invoke({'messages': [message]})

response['messages']

[HumanMessage(content='Who is the first person to walk on the moon?', additional_kwargs={}, response_metadata={}, id='67eccd49-cc83-41cf-83cd-e9cbe4e8a268'),
 AIMessage(content='The first person to walk on the moon was **Neil Armstrong**.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019af767-d9cd-76e2-a425-d29b77508726-0', usage_metadata={'input_tokens': 12, 'output_tokens': 13, 'total_tokens': 25, 'input_token_details': {'cache_read': 0}})]

## Step 7: Interactive Conversation Loop

Run an interactive chatbot session:
- Continuously prompts for user input
- Maintains conversation history in the state
- Each new message is appended to the existing conversation
- Type 'exit' or 'quit' to end the conversation
- The LLM can reference previous messages for context-aware responses

In [22]:
state = None

while True:
    in_message = input("User: ")
    if in_message.lower() in ['exit', 'quit']:
        break

    if state is None:
        state = {
            'messages': [{"role": "user", "content": in_message}]
        }
    else:
        state['messages'].append({"role": "user", "content": in_message})
    
    state = graph.invoke(state)

    print("Bot:", state['messages'][-1].content)

Bot: Neil Armstrong
Bot: July 20, 1969
Bot: July 20, 1969
